In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
from utils import create_wfv_splits

In [2]:
training = True
testing = True
n_splits = 3
test_idx = 2
mean_types = ["global", "hour", "day", "buurt", "buurt_hour", "buurt_dayhour"]

target_df = pd.read_pickle("data/PARKING/raw/target_df_aantal_cleaned_new.pkl").dropna(how="all")
wfv_splits, train_size, test_size = create_wfv_splits(target_df, n_splits)
target_df.index = [target_df.index.dayofweek, target_df.index.hour]

mean_df = target_df
mean_df = mean_df.groupby(mean_df.index).mean()
mean_df = mean_df.reindex(target_df.index)
mean_df

buurtcode                a00a      a00b      a00c      a00d      a00e  \
scnMoment scnMoment                                                     
6         9               NaN       NaN       NaN       NaN       NaN   
          12         0.690501  0.508266  0.771743  0.894849  0.830548   
          13         0.662869  0.461740  0.850733  0.895590  0.861088   
          14         0.689147  0.498138  0.846392  0.652174  0.839159   
          15         0.662592  0.451184  0.823421  0.859089  0.838432   
...                       ...       ...       ...       ...       ...   
0         13         0.703500  0.582238  0.844373  0.890888  0.876578   
          14         0.692278       NaN  0.830617  0.608696  0.883879   
          15         0.655532  0.534554  0.812028  0.812762  0.817295   
          16         0.511715  0.573389  0.775057  0.787677  0.810272   
          17         0.433136  0.440723  0.784500  0.774781  0.776619   

buurtcode                a01a      a01b  a01c      a01d  a01e  ...  t96d  \
scnMoment scnMoment                                            ...         
6         9               NaN       NaN   NaN       NaN   NaN  ...   NaN   
          12              NaN  0.761412   NaN  0.736965   NaN  ...   NaN   
          13              NaN  0.764746   NaN  0.780746   NaN  ...   NaN   
          14         0.058824  0.780930   NaN  0.801797   NaN  ...   NaN   
          15              NaN  0.778740   NaN  0.743035   NaN  ...   NaN   
...                       ...       ...   ...       ...   ...  ...   ...   
0         13              NaN  0.775580   NaN  0.813641   NaN  ...   NaN   
          14              NaN  0.801547   NaN  0.764710   NaN  ...   NaN   
          15              NaN  0.728225   NaN  0.701877   NaN  ...   NaN   
          16              NaN  0.755318   NaN  0.717493   NaN  ...   NaN   
          17              NaN  0.718340   NaN  0.689767   NaN  ...   NaN   

buurtcode            t96e  t96f  t96g  t97a  t97b  t97c  t97d  t98a  t98b  
scnMoment scnMoment                                                        
6         9           NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
          12          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
          13          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
          14          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
          15          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
...                   ...   ...   ...   ...   ...   ...   ...   ...   ...  
0         13          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
          14          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
          15          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
          16          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
          17          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  

[12955 rows x 481 columns]

In [3]:
wfv_splits


array([[range(0, 9751), range(9751, 10696)],
       [range(0, 10696), range(10696, 11824)],
       [range(0, 11824), range(11824, 12955)]], dtype=object)

In [4]:
train_losses_df = pd.DataFrame(index=range(len(wfv_splits)), columns=mean_types, dtype = np.float64)
val_losses_df = pd.DataFrame(index=range(len(wfv_splits)), columns=mean_types, dtype = np.float64)

for mean_num, mean_type in enumerate(mean_types):
    target_df = pd.read_pickle("data/PARKING/raw/target_df_aantal_cleaned_new.pkl").dropna(how="all")
    
    if mean_type == "hour" or mean_type == "buurt_hour":
        target_df.index = target_df.index.hour
    elif mean_type == "day":
        target_df.index = target_df.index.dayofweek
    elif mean_type == "buurt_dayhour":
        target_df.index = [target_df.index.dayofweek, target_df.index.hour]
    results = []

    for split_idx, (train_index, val_index) in enumerate(wfv_splits):
        if (not training and split_idx < test_idx) or (not testing and split_idx >= test_idx): continue
            
        X_train = target_df.iloc[train_index]
        X_val = target_df.iloc[val_index]

        if mean_type == "buurt_hour" or mean_type == "buurt_dayhour":
            mean_df = X_train.groupby(X_train.index).mean()
            mean_df = mean_df.fillna(mean_df.mean())
            train_err_df = X_train - mean_df.reindex(X_train.index)
            val_err_df = X_val - mean_df.reindex(X_val.index)
        else:
            if mean_type == "hour" or mean_type == "day":
                idx, values = zip(*[(x, np.nanmean(y)) for x, y in X_train.groupby(X_train.index)])
                train_err_df = X_train.sub(pd.Series(values, idx).reindex(X_train.index), axis="index")
                val_err_df = X_val.sub(pd.Series(values, idx).reindex(X_val.index), axis="index")
            else:
                if mean_type == "global":
                    mean_df = np.nanmean(X_train)
                    val_err_df = X_val - mean_df
                elif mean_type == "buurt":
                    mean_df = X_train.mean()
                    val_err_df = X_val - mean_df.fillna(mean_df.mean())
                train_err_df = X_train - mean_df
                

        train_losses_df.iloc[split_idx, mean_num] = (np.nanmean(train_err_df ** 2)) ** 0.5
        val_losses_df.iloc[split_idx, mean_num] = (np.nanmean(val_err_df ** 2)) ** 0.5

if training:
    print(" "*18, 'train', " "*4, 'val')
    for mean_num, mean_type in enumerate(mean_types):
        print('{:<14s} | {:<10f} | {:<10f} |'.format(mean_type, np.average(train_losses_df.iloc[:test_idx, mean_num], weights = train_size[:test_idx]), 
                                                                np.average(val_losses_df.iloc[:test_idx, mean_num], weights = test_size[:test_idx])))

if testing:
    print("\n", " "*18, 'train', " "*4, 'test')
    for mean_num, mean_type in enumerate(mean_types):
        print('{:<14s} | {:<10f} | {:<10f} |'.format(mean_type, np.average(train_losses_df.iloc[test_idx:, mean_num], weights = train_size[test_idx:]), 
                                                                np.average(val_losses_df.iloc[test_idx:, mean_num], weights = test_size[test_idx:])))

                   train      val
global         | 0.188593   | 0.191064   |
hour           | 0.180692   | 0.183898   |
day            | 0.188357   | 0.191050   |
buurt          | 0.110265   | 0.118706   |
buurt_hour     | 0.091821   | 0.105004   |
buurt_dayhour  | 0.083832   | 0.105532   |

                    train      test
global         | 0.188973   | 0.198726   |
hour           | 0.181165   | 0.194667   |
day            | 0.188756   | 0.198265   |
buurt          | 0.110666   | 0.115227   |
buurt_hour     | 0.092923   | 0.105011   |
buurt_dayhour  | 0.085343   | 0.103991   |


In [5]:
train_losses_df

,global,hour,day,buurt,buurt_hour,buurt_dayhour
0,0.188587,0.180663,0.188332,0.109780,0.090949,0.082780
1,0.188599,0.180718,0.188380,0.110705,0.092612,0.084786
2,0.188973,0.181165,0.188756,0.110666,0.092923,0.085343


In [6]:
val_losses_df

,global,hour,day,buurt,buurt_hour,buurt_dayhour
0,0.189302,0.181830,0.189478,0.126628,0.112171,0.113676
1,0.192696,0.185815,0.192507,0.111362,0.098360,0.097983
2,0.198726,0.194667,0.198265,0.115227,0.105011,0.103991


In [7]:
train_losses_df.iloc[test_idx:, mean_num]


2    0.085343
Name: buurt_dayhour, dtype: float64

In [8]:
np.average(train_losses_df.iloc[:test_idx, mean_num], weights = train_size[:test_idx])


0.08383226960649302

## DATA GCN

In [9]:
buurt_df = pd.read_pickle("data/PARKING/raw/buurt_polygons.pkl")
buurt_df.head()

,buurtcode,polygon,centroid,norm_centroid
221,a00a,"POLYGON ((122045.505 487745.336, 122026.815 48...",POINT (121841.64116914148 487673.3461266836),POINT (0.027989097150902 0.1717701745775366)
318,a00b,"POLYGON ((121782.088 487542.257, 121770.331 48...",POINT (121559.47899824797 487437.3476033915),POINT (0.0004469220902971 0.1444060134852146)
76,a00c,"POLYGON ((121885.329 487401.961, 121908.361 48...",POINT (121735.38780294551 487327.7959828982),POINT (0.0176175834810113 0.1317034416670286)
275,a00d,"POLYGON ((121539.554 487222.451, 121467.922 48...",POINT (121391.6158944025 487112.2825822399),POINT (-0.0159383891570899 0.106714542029424)
23,a00e,"POLYGON ((121734.944 487124.933, 121620.093 48...",POINT (121517.8818392263 486964.5833259047),POINT (-0.0036134245967536 0.0895887304472436)


In [10]:
def normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx

def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)

In [11]:
from itertools import repeat
import scipy.sparse as sp
import torch
import torch.nn as nn

# buurt_df = buurt_df.set_index('buurtcode')
idx_map = {j: i for i, j in enumerate(buurt_df.index)}
buurt_gdf = gpd.GeoDataFrame(geometry=buurt_df.polygon)

edges_unordered = np.concatenate([list(zip(buurt_gdf.index[buurt_gdf.touches(row.geometry)], repeat(code))) 
                            for code, row in buurt_gdf.iterrows()])
edges = np.array(list(map(idx_map.get, edges_unordered.flatten()))).reshape(edges_unordered.shape)

adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])),
                    shape=(len(buurt_gdf), len(buurt_gdf)))
adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)
adj = normalize(adj + sp.eye(adj.shape[0]))
adj = sparse_mx_to_torch_sparse_tensor(adj)

# GCN

In [12]:
class GraphConvolution(nn.Module):
    """
    Simple GCN layer, similar to https://arxiv.org/abs/1609.02907
    """

    def __init__(self, in_features, out_features, bias=True):
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.FloatTensor(in_features, out_features))
        if bias:
            self.bias = Parameter(torch.FloatTensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input, adj):
        support = torch.mm(input, self.weight)
        output = torch.spmm(adj, support)
        if self.bias is not None:
            return output + self.bias
        else:
            return output

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ')'
    
class GCN(torch.nn.Module):
    def __init__(self, nfeat, nhid, nout, dropout):
        super(GCN, self).__init__()
        self.gc1 = GraphConvolution(nfeat, nhid)
        self.gc2 = GraphConvolution(nhid, nout)
        self.dropout = dropout

    def forward(self, x, adj):
        x = F.relu(self.gc1(x, adj))
        x = F.dropout(x, self.dropout, training=self.training)
        x = self.gc2(x, adj)
        return x

In [13]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
adj = adj.to(device)
max_epochs = 50
max_folds = 3
hidden_num = 50
output_dim = 1
target_df = pd.read_pickle("../../data/processed_targets/target_df_aantal_cleaned_new.pkl").dropna(how="all")

input_list = ["cyclical_hour", "cyclical_day", "centroid", "bbga", "npr", "historical"]
all_combinations = []
for r in range(1, len(input_list) + 1):
    all_combinations += list(itertools.combinations(input_list, r))
    
for input_types in [["cyclical_hour", "cyclical_day", "centroid", "bbga", "npr", "historical"]]:
    
    train_losses_df = pd.DataFrame(index=range(max_epochs), columns=range(max_folds))
    test_losses_df = pd.DataFrame(index=range(max_epochs), columns=range(max_folds))
    
    train_index = int(len(target_df)*0.70)
    val_index = int(len(target_df)*0.80)
    training_set = Dataset(input_types, target_df.iloc[:train_index])
    val_set = Dataset(input_types, target_df.iloc[train_index:val_index])
    
#     tscv = TimeSeriesSplit(n_splits=max_folds)
#     for nfold, (train_index, test_index) in enumerate(tscv.split(target_df)):

    for nfold in range(max_folds):
        if nfold == 0: print(input_types, ":", training_set[0][0].shape)
    
        model = GCN(nfeat=training_set[0][0].shape[-1], nhid=hidden_num, nout=output_dim, dropout=0.2).to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay = 1e-4)

        for epoch in range(max_epochs):
            train_losses, test_losses = [], []

            training_generator = torch.utils.data.DataLoader(training_set)
            test_generator = torch.utils.data.DataLoader(val_set)

            model.train()
            for x, y in training_generator:
                mask = ~torch.isnan(y[0])
                x, y, mask = x[0].to(device), y[0].to(device), mask.to(device)
                optimizer.zero_grad()
                outputs = model(x, adj).flatten()
                loss = loss_fn(outputs[mask], y[mask])
                loss.backward()
                optimizer.step()
                train_losses.append(loss.detach().cpu().item())
            train_losses_df.iloc[epoch, nfold] = np.mean(train_losses)

            model.eval()
            for x, y in test_generator:
                mask = ~torch.isnan(y[0])
                x, y, mask = x[0].to(device), y[0].to(device), mask.to(device)
                outputs = model(x, adj).flatten()
                loss = loss_fn(outputs[mask], y[mask])
                test_losses.append(loss.detach().cpu().item())
            test_losses_df.iloc[epoch, nfold] = np.mean(test_losses)
            print("epoch:", epoch)
            print("training loss:", train_losses_df.iloc[epoch, nfold])
            print("test loss:", test_losses_df.iloc[epoch, nfold])
        print("nfold:", nfold)
        print("training loss:", train_losses_df.iloc[epoch, nfold])
        print("test loss:", test_losses_df.iloc[epoch, nfold])
    print("mean training loss:", train_losses_df.iloc[-1,:].mean())
    print("mean testing loss:", test_losses_df.iloc[-1,:].mean())

    train_losses_df.plot()
    test_losses_df.plot()
    plt.show()

FileNotFoundError: [Errno 2] No such file or directory: '../../data/processed_targets/target_df_aantal_cleaned_new.pkl'

In [ ]:
class Dataset(torch.utils.data.Dataset):
    
    def __init__(self, input_types, target_df):
        'Initialization'
        bbga_df = pd.read_pickle("../../data/processed_inputs/bbga_pca.pkl")
        spatial_polygons = pd.read_pickle("../../data/processed_inputs/buurt_polygons.pkl")
        
        self.input_types = input_types
        self.batch_size = target_df.shape[-1]
        self.time_idx = target_df.index
        self.time_range = pd.date_range(start=self.time_idx[0], end=self.time_idx[-1], freq='H')
        self.hour_one_hot = torch.nn.functional.one_hot(torch.LongTensor(target_df.index.hour)).float()
        self.day_one_hot = torch.nn.functional.one_hot(torch.LongTensor(target_df.index.dayofweek)).float()
        self.spatial_one_hot = torch.eye(self.batch_size)
        self.spatial_centroid = torch.Tensor(np.vstack([[p.x, p.y] for p in spatial_polygons['norm_centroid']]))
        self.bbga_data = torch.Tensor(bbga_df.loc[2018].T.values)
        self.cyclical_hour = torch.Tensor([np.sin(2*np.pi*target_df.index.hour/24), np.cos(2*np.pi*target_df.index.hour/24)]).T
        self.cyclical_day = torch.Tensor([np.sin(2*np.pi*target_df.index.dayofweek/24), np.cos(2*np.pi*target_df.index.dayofweek/24)]).T

        self.historical_values = target_df.reindex(self.time_range).shift()
        
        if "npr" in input_types:
            self.npr_occ = pd.read_csv("../../data/raw_buurt_info/NPR_PRC_Stacked_Occupation.csv", sep = ';')
            self.npr_occ["buurtcode"] = self.npr_occ["buurtcode"].str.lower()
            self.npr_occ = pd.melt(self.npr_occ, id_vars=['buurtcode', 'B_TYD_V_RECHT'], value_vars=self.npr_occ.columns[3:], value_name='count')
            self.npr_occ['interval_start'] = pd.to_datetime(self.npr_occ['B_TYD_V_RECHT'] + '-' + self.npr_occ['variable'].str[5:], format='%Y-%m-%d-%H')
            self.npr_occ = self.npr_occ.groupby(['buurtcode', 'interval_start']).sum().unstack(level=0)
            self.npr_occ = self.npr_occ.droplevel(level=0, axis=1)
            self.npr_occ = self.npr_occ.shift().reindex(self.time_range, columns=target_df.columns)
        
        self.y = torch.Tensor(target_df.values)
        
    def __len__(self):
        'Denotes the total number of samples'
        return len(self.y)
    
    def __getitem__(self, index):
        'Generates one sample of data'
        time_idx = self.time_idx[index]
        feature_list = []
        for input_type in self.input_types:
            if input_type == "global":
                feature_list.append(torch.ones(self.batch_size, 1))
            if input_type == "hour":
                feature_list.append(self.hour_one_hot[index].expand(self.batch_size, -1))
            if input_type == "day":
                feature_list.append(self.day_one_hot[index].expand(self.batch_size, -1))
            if input_type == "cyclical_hour":
                feature_list.append(self.cyclical_hour[index].expand(self.batch_size, -1))
            if input_type == "cyclical_day":
                feature_list.append(self.cyclical_day[index].expand(self.batch_size, -1))
            if input_type == "neighbourhood":
                feature_list.append(self.spatial_one_hot)
            if input_type == "centroid":
                feature_list.append(self.spatial_centroid)
            if input_type == "bbga":
                feature_list.append(self.bbga_data)
            if input_type == "historical":
                feature_list.append(torch.Tensor(np.stack((self.historical_values.loc[time_idx].fillna(0), self.historical_values.loc[time_idx].notnull()), axis=1)))
            if input_type == "npr":
                feature_list.append(torch.Tensor(np.stack((self.npr_occ.loc[time_idx].fillna(0), self.npr_occ.loc[time_idx].notnull()), axis=1)))
        x = torch.cat((feature_list), -1)
        y = self.y[index]
        mask = ~torch.isnan(y)
        return x[mask], y[mask]
    
target_df = pd.read_pickle("../../data/processed_targets/target_df_aantal_cleaned_new.pkl").dropna(how="all")